# Bemenet feldolgozása

Ezen notebook kiegészítséként szolgál a dolgozat 3. fejezetéhez.

## Tartalomjegyzék
1. [Természetes nyelvi szöveg feldolgozása](#Természetes-nyelvi-szöveg-feldolgozása)
2. [Strukturált adatok megalkotása](#Strukturált-adatok-megalkotása)

## Természetes nyelvi szöveg feldolgozása

A bemeneti mondat egy egyszerű normalizálási folyamaton megy keresztül, hogy a további feldolgozást megkönnyítse. A normalizálási folyamat a következő:


- Írásjelek eltávolítása
- A szöveg kisbetűssé tétele
- A szavak kigyűjtése (tokenek megalkotása)
- A százalékos értékeket jelző tokenek szerializálása


In [1]:
# Írásjelek eltávolítása
def remove_punctuations(input_text):
    punctuations = [".", ",", ":", ";", "!", "?"]

    result_text = input_text
    for punctuation in punctuations:
        result_text = result_text.replace(punctuation, "")
    return result_text

# Szöveg kisbetűssé tétele
def make_lowercase(input_text):
    lowercase_text = input_text.lower()
    return lowercase_text

# A szavak kigyűjtése (tokenek megalkotása)
def tokenize_text(input_text):
    tokens = input_text.split()
    return tokens

# A százalékos értékeket jelző tokenek szerializálása
def serialize_percentage_tokens(tokens):
    serialized_tokens = tokens.copy()
    percentage_symbols = ["%", "százalék"]
    unused_tokens = []
    for i in range(len(serialized_tokens)):
        for percentage_symbol in percentage_symbols:
            if percentage_symbol == serialized_tokens[i]:
                if(serialized_tokens[i-1].isnumeric()):
                    serialized_tokens[i-1] = serialized_tokens[i-1]+"%"
                unused_tokens.append(serialized_tokens[i])

    for i in unused_tokens:
        serialized_tokens.remove(i)

    return serialized_tokens

In [2]:
def normalize_text(input_text):
    normalized_text = remove_punctuations(input_text)
    normalized_text = make_lowercase(normalized_text)
    tokens = tokenize_text(normalized_text)
    tokens = serialize_percentage_tokens(tokens)

    return tokens

In [3]:
# A bemeneti mondat normalizálása és tokenek megalkotása
input_sentence = "A képen megfigyelhető egy macska, amely 20%-ban eb!"
tokens = normalize_text(input_sentence)

tokens

['a', 'képen', 'megfigyelhető', 'egy', 'macska', 'amely', '20%-ban', 'eb']

## Strukturált adatok megalkotása

Ha a bemeneti mondat normalizálásra került, úgy előállítható belőle a strukturált adat, amely jelen esetben python dictionary formátumban kerül megalkotásra.

A dolgozatom ezen része nem terjed ki olyan részletesen a Természetes Nyelvi Szövegfeldolgozásra, a rendelkezésre álló adatok hiányában egy egyszerűbb megoldást választottam, amely szinonima szótár segítségével osztályozza a megfelelő tokeneket.

In [4]:
# Szinonímák a https://szinonimaszotar.hu weboldalról

synonyms = {
    "cat": ["macska", "cica", "cicus", "cirmos", "cicamica", "kandúr", "macsi", "macsek", "cicó", "cila", "macs", "cilamila", "macskusz", "cicuska", "ciculi", "cirmi", "cicuka", "cicmic", "cic", "ciccancs", "kismacska", "mafka", "kiscica"],
    "dog": ["kutya", "eb", "kutyus", "kutyuli", "blöki", "véreb", "öleb", "kutyuska", "csahos", "házőrző", "kutyi", "bolhazsák", "kutyi-mutyi", "kutyuli-mutyuli", "vahúr", "négylábú", "kutyu", "kutyóka"],
    "wild": ["vadállat", "vad", "szörny", "szörnyeteg", "fenevad", "bestia", "dúvad", "fúria"]
}

In [5]:
# Tokenek tartalmazásának vizsgálata

def compare_two_tokens(token_a, token_b):
    best_match = 0.0
    offset = 0
    while(offset <= len(token_b) - len(token_a)):
        count = 0
        for i in range(len(token_a)):
            if(token_a[i] == token_b[offset + i]):
                count += 1
            else:
                break
        match_value = count/len(token_a)

        if(match_value > best_match):
            best_match = match_value
        offset += 1
    return best_match

def find_best_match_for_token_among_tokens(test_token, tokens):
    best_match = 0.0
    for token in tokens:
        match_value = compare_two_tokens(token, test_token)
        if(match_value > best_match):
            best_match = match_value
    return best_match

In [6]:
# Osztályok azonosítása a szinonímák tartalmazása szerint
def find_classes(tokens, threshold):
    result_classes = []
    for i in range(len(tokens)):
        for key in synonyms.keys():
            match_value =\
                find_best_match_for_token_among_tokens(
                    tokens[i],
                    synonyms[key]
                )
            if(match_value >= threshold):
                found_class = {"token_id" : i, "class" : key}
                if not found_class in result_classes:
                    result_classes.append(found_class)
    return result_classes

# A bemenetben megadott százalékos értékek kikeresése a tokenekből
def find_percentage_values(tokens):
    res = []
    for i in range(len(tokens)):
        if "%" in tokens[i]:
            prob = int(tokens[i].split("%")[0]) / 100
            found_class = {"token_id" : i, "value" : prob}
            if not found_class in res:
                res.append(found_class)
    return res

In [7]:
# A kimeneti python dictionary elkészítése a talált osztályok és értékek alapján
def make_dict_from_classes(res_classes, res_probs):
    class_names = synonyms.keys()
    base_data = dict()

    for class_entity in class_names:
        base_data[class_entity] = 0.0

    if(len(res_probs) > 0):
        found_classes = []
        sums_of_prod = 0
        for i in range(len(res_probs)):
            for j in range(len(res_classes)):
                if(res_probs[i]["token_id"] < res_classes[j]["token_id"]):
                    base_data[res_classes[j]['class']] = res_probs[i]['value']
                    sums_of_prod += res_probs[i]['value']
                    found_classes.append(res_classes[j])
                    break
        if(sums_of_prod != 1):
            if(len(found_classes) < len(res_classes)):
                for found_class in found_classes:
                    res_classes.remove(found_class)
                for res_class in res_classes:
                    base_data[res_class['class']] = (1 - sums_of_prod) / len(res_classes)
            elif(len(found_classes) == len(res_classes)):
                remaining_classes = list(class_names)
                for found_class in found_classes:
                    remaining_classes.remove(found_class['class'])
                for remaining_class in remaining_classes:
                    base_data[remaining_class] = (1 - sums_of_prod) / len(remaining_classes)
        
    else:
        for found_class in map(lambda x: x["class"], res_classes):
            base_data[found_class] = 1 / len(res_classes)
    return base_data

In [8]:
def make_structured_data(tokens, threshold):
    result_classes = find_classes(tokens, threshold)
    result_percentages = find_percentage_values(tokens)
        
    return make_dict_from_classes(result_classes, result_percentages)

In [9]:
structured = make_structured_data(tokens, threshold=0.8)

print(f"Bementi mondat: {input_sentence}\n")
print(f"Tokenek: {tokens}\n")
print(f"Strukturált adat: {structured}")

Bementi mondat: A képen megfigyelhető egy macska, amely 20%-ban eb!

Tokenek: ['a', 'képen', 'megfigyelhető', 'egy', 'macska', 'amely', '20%-ban', 'eb']

Strukturált adat: {'cat': 0.8, 'dog': 0.2, 'wild': 0.0}
